# Accessing Databases via Web APIs
* * * * *

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

## The New York Times API: All the Data Thats Fit to Query

### 1. Establishing the Connection

Put the API Key you just created in the `api_key` variable in the cell below

In [ ]:
#put your key here
api_key = ""

To access the NYTimes' databases we'll be using a third-party library called [pynytimes](https://github.com/michadenheijer/pynytimes). This package provides an easy to use tool for accessing the wealth of data hosted by the Times.

To install the library, follow these instructions taken from their github repo.

## Installation

There are multiple options to install and ugprade pynytimes, but the easiest is by just installing it using ```pip``` (or ```pip3```).
### Linux and Mac

```bash
pip install --upgrade pynytimes
```

### Windows

```shell
python -m pip install --upgrade pynytimes
```

### Development

You can also install ```pynytimes``` manually from GitHub itself. This can be done by cloning this repository first, and then installing it using Python. *This might install an unreleased version, installation using this method is only advised if you want to modify the code or help maintain this library.*

```bash
git clone https://github.com/michadenheijer/pynytimes.git
cd pynytimes
python setup.py install
```

### Older Python versions

The current version of ```pynytimes``` only supports the most recent Python versions (3.8, 3.9, and 3.10) however you still might be running older versions of Python. Luckily most of ```pynytimes``` features are still available. In the table below you can see which version of ```pynytimes``` still supports your Python version.

| Python version | ```pynytimes``` version | Missing features                             |
|----------------|-------------------------|----------------------------------------------|
| 3.7            | ```0.7.0```             | Some type hints, small bugfixes              |
| 3.6            | ```0.6.1```             | Type hints, small bugfixes, ```with``` usage |
| 3.5            | ```0.4.2```             | Times Tags, no date parsing                  |

You can install an older version by ```pip install --upgrade pynytimes==0.7.0```.

In [ ]:
!pip install pynytimes

Once the package installed, let's go ahead import the library and initialize a connection to their servers using our api keys.

In [ ]:
#Import the NYTAPI object which we'll use to access the API
from pynytimes import NYTAPI

In [ ]:
#intialize nyt api class using your api key
nyt = NYTAPI(api_key, parse_dates=True)

Ta-da! We are now ready to make some API calls!!

### 2. Making API Calls
*****

Here's comes the fun part. Now that we've established a connection to New York Times' rich database, let's go over what kind of data and privileges we have access to.

### APIs

[Here is the collection of the APIs the NYT gives us.](https://developer.nytimes.com/apis)

- [Top stories](https://developer.nytimes.com/docs/top-stories-product/1/overview): Returns an array of articles currently on the specified section 
- [Most viewed/shared articles](https://developer.nytimes.com/docs/most-popular-product/1/overview): Provides services for getting the most popular articles on NYTimes.com based on emails, shares, or views.
- [Article search](https://developer.nytimes.com/docs/articlesearch-product/1/overview): Look up articles by keyword. You can refine your search using filters and facets.
- [Books](https://developer.nytimes.com/docs/books-product/1/overview): Provides information about book reviews and The New York Times Best Sellers lists.
- [Movie reviews](https://developer.nytimes.com/docs/movie-reviews-api/1/overview): Search movie reviews by keyword and opening date and filter by Critics' Picks.
- [Times Wire](https://developer.nytimes.com/docs/timeswire-product/1/overview): Get links and metadata for Times' articles as soon as they are published on NYTimes.com. The Times Newswire API provides an up-to-the-minute stream of published articles.
- [Tag query (TimesTags)](https://developer.nytimes.com/docs/timestags-product/1/overview): Provide a string of characters and the service returns a ranked list of suggested terms.
- [Archive metadata](https://developer.nytimes.com/docs/archive-product/1/overview): Returns an array of NYT articles for a given month, going back to 1851.

In this workshop we go over a few of the APIs and do some light data analysis of the data we pull.

**Top Stories**

Let's see what are the top stories of the day. To do that all we have to do is call a single method on the `nyt` object

In [ ]:
# Get all the top stories from the home page
top_stories = nyt.top_stories()

print("top_stories is a list of length {}".format(len(top_stories)))

The `top_stories` method has a single paramater called `section` parameter defaults to "home".

In [ ]:
#Preview the output
top_stories[:3]

This is pretty typical output for data pulled from an API. We are looking at a list of nested JSON dictionaries.

When working with a new api, a good way to establish an understanding of the data is to inspect a single object in the collection. Let's grab the first story in the array and inspect it's attributes and data.

In [ ]:
top_story = top_stories[0]
top_story

We are provided a diverse collection of data for the article ranging from the expected (title, author, section) and to NLP-derived information such as named entities. And FYI does not provide the full article in their API.

**Does anything about the data stand out to you? What bits of information could be useful to you and your research needs?**

If we are interested in a specific section then we can pass in one of the following tags into the `section` parameter:


```arts```, ```automobiles```, ```books```, ```business```, ```fashion```, ```food```, ```health```, ```home```, ```insider```, ```magazine```, ```movies```, ```national```, ```nyregion```, ```obituaries```, ```opinion```, ```politics```, ```realestate```, ```science```, ```sports```, ```sundayreview```, ```technology```, ```theater```, ```tmagazine```, ```travel```, ```upshot```, and ```world```.


#### Challenge 1: Find the top stories for a section

- Choose 2 sections, grab their top stories and collect them into lists.
- Tell us how many stories are each in section and what is the title of the first story in each list.

In [ ]:
#solution

#### Organizing the API results into a Pandas DataFrame

In order to conduct any sort of analysis we need to convert the list of JSON data to a Pandas dataframe. Pandas allows use to simply pass in the array of article data and produce a nice clean table in one line of code. 

First let's see what happens when we pass in `top_stories` to `pd.json_normalize`

In [ ]:
#Convert to DataFrmae
df = pd.json_normalize(top_stories)
#View first 5 rows
df.head()

In [ ]:
#Inspect metadata
df.info()

For the most part it does a good job of producing a table where
- The columns correspond with the JSON dictionary keys from our API call.
- The number of rows matches the number of articles.
- Each cell holds the corresponding value found under that article's dictionary key.

However there is one issue and that can be found in the multimedia column

In [ ]:
#Grab the multimedia column data
multimedia = df["multimedia"]
multimedia.head()

In [ ]:
#first row
multimedia.iloc[0]

The data in the multimedia column is what's referred to as a nested list. An issue may arise if we'd like to treat attributes in an article's `multimedia` data as distinct columns. The `json_normalize` method has parameters that get around this.

In [ ]:
#Call the json_normalize method by setting record_path to multimedia
#Set record_prefix to "multimedia_" to identify the data under multimedia
#Set meta to a list of columns from the original dataframe cols
cols = df.columns.tolist()
df = pd.json_normalize(top_stories, record_path = "multimedia", record_prefix= "multimedia_", meta=cols)
df.head()

All the information from multimedias has been restructed to be on the same level as the rest of the data.

In [ ]:
#save data to csv file
df.to_csv("top_nyt_articles.csv")

#### Most viewed and shared articles

Retrieving the most viewed and shared articles is also quite simple. The `days` parameter returns the most popular articles based on the last N days, only allowed to pass in 1, 7, or 30.

In [ ]:
#Retrieve the most viewed articles for today. Days defaults to 1
most_viewed_today = nyt.most_viewed()
most_viewed_today[0]

In [ ]:
len(most_viewed_today)

For this piece of data, we can consult a guide or what's known as a schema to understand the information at our finger tips.

The [Most Viewed Schema](https://developer.nytimes.com/docs/most-popular-product/1/types/ViewedArticle) can answer any questions we may have about this article's data.

| Attribute      | Data Type | Definition      |
| ----------- | ----------- | ----------- |
| url      | string       | Article's URL.       |
| adx_keywords   | string        | Semicolon separated list of keywords.        |
| column   | string        | Deprecated. Set to null.        |
| section   | string        | Article's section (e.g. Sports).        |
| byline   | string        | Article's byline (e.g. By Thomas L. Friedman).        |
| type   | string        | Asset type (e.g. Article, Interactive, ...).        |
| title   | string        | Article's headline (e.g. When the Cellos Play, the Cows Come Home).        |
| abstract   | string        | Brief summary of the article.|
| published_date   | string        | When the article was published on the web (e.g. 2021-04-19).        |
| source   | string        | Publisher (e.g. New York Times).        |
| id   | integer        | Asset ID number (e.g. 100000007772696).        |
| asset_id   | integer        | Asset ID number (e.g. 100000007772696).        |
| des_facet   | array        | Array of description facets (e.g. Quarantine (Life and Culture)).        |
| org_facet   | array        | Array of organization facets (e.g. Sullivan Street Bakery).        |
| per_facet   | array        | Array of person facets (e.g. Bittman, Mark).        |
| geo_facet   | array        | Array of geographic facets (e.g. Canada).        |
| media   | array        | Array of images.        |
| media.type   | string        | Asset type (e.g. image).        |
| media.subtype   | string        | Asset subtype (e.g. photo).        |
| media.caption   | string        | Media caption        |
| media.copyright   | string        | Media credit        |
| media.approved_for_syndication   | boolean        | Whether media is approved for syndication.        |
| media.media-metadata   | array        | Media metadata (url, width, height, ...).        |
| media.media-metadata.url   | string        | Image's URL.        |
| media.media-metadata.format   | string        | Image's crop name     |
| media.media-metadata.height   | integer        | Image's height |
| media.media-metadata.width   | integer        | Image's width      |

To pull most popular articles for the past weekend and month, we pass the numbers 7 or 30 into `days`

In [ ]:
most_viewed_week = nyt.most_viewed(days=7)
most_viewed_month = nyt.most_viewed(days=30)

Most viewed article of the last week

In [ ]:
most_viewed_week[0]

Most viewed article of the last month

In [ ]:
most_viewed_month[0]

#### Challenge 2: Find the most shared stories via email and facebook.

The `most_shared` method is similiar to `most_viewed` except that has an argument called `method` which is used to show the most shared articles using 'email' or 'facebook'.

- Grab the most shared articles for both methods for the past month.
- How many articles show up in both lists? (Hint: use the 'uri' key)
- Bonus. Use the [Shared Article](https://developer.nytimes.com/docs/most-popular-product/1/types/SharedArticle) schema table to help you answer a question you may have about the data.

| Attribute      | Data Type | Definition      |
| ----------- | ----------- | ----------- |
| url      | string       | Article's URL.       |
| adx_keywords   | string        | Semicolon separated list of keywords.        |
| subsection   | string        | Article's subsection (e.g. Politics). Can be empty |
| column   | string        | Deprecated. Set to null.        |
| eta_id   | integer        | Deprecated. Set to 0.|
| section   | string        | Article's section (e.g. Sports).        |
| id   | integer        | Asset ID number (e.g. 100000007772696).        |
| asset_id   | integer        | Asset ID number (e.g. 100000007772696).        |
| nytdsection   | string        | Article's section|
| byline   | string        | Article's byline (e.g. By Thomas L. Friedman).        |
| type   | string        | Asset type (e.g. Article, Interactive, ...).        |
| title   | string        | Article's headline (e.g. When the Cellos Play, the Cows Come Home).        |
| abstract   | string        | Brief summary of the article.|
| published_date   | string        | When the article was published on the web (e.g. 2021-04-19).        |
| source   | string        | Publisher (e.g. New York Times).        |
| updated   | string        | When the article was last updated (e.g. 2021-05-12 06:32:03).|
| des_facet   | array        | Array of description facets (e.g. Quarantine (Life and Culture)).        |
| org_facet   | array        | Array of organization facets (e.g. Sullivan Street Bakery).        |
| per_facet   | array        | Array of person facets (e.g. Bittman, Mark).        |
| geo_facet   | array        | Array of geographic facets (e.g. Canada).        |
| media   | array        | Array of images.        |
| media.type   | string        | Asset type (e.g. image).        |
| media.subtype   | string        | Asset subtype (e.g. photo).        |
| media.caption   | string        | Media caption        |
| media.copyright   | string        | Media credit        |
| media.approved_for_syndication   | boolean        | Whether media is approved for syndication.        |
| media.media-metadata   | array        | Media metadata (url, width, height, ...).        |
| media.media-metadata.url   | string        | Image's URL.        |
| media.media-metadata.format   | string        | Image's crop name     |
| media.media-metadata.height   | integer        | Image's height |
| media.media-metadata.width   | integer        | Image's width      |
| uri   | string        | An article's globally unique identifier.      |

In [1]:
#solution

#### Article Search

Time to take it up a notch and use the search API to retrieve a set of articles about a particular topic in a chosen period of time.

Arguments:

`query`: The search query

`results`: Number of articles returned. Default = 10.

Pull the most recent 20 articles about Elon Musk.

In [ ]:

articles = nyt.article_search(
    query = "Elon Musk",
    results = 20)

Assign the data in the first item of `articles` to a variable

In [ ]:
article = articles[0].copy()

We need to drop the `multimedia` key to reduce the amount of clutter on our screens

In [ ]:
del article["multimedia"]

In [ ]:
#View article details

In [ ]:
article

You'll notice by now that not all article data comes in the same format. Data from the search API is presented differently from that of the most_viewed and top_stories APIs.

There are schemas for the above data but unfortunately they do not have definitions.

- [Article Schema](https://developer.nytimes.com/docs/articlesearch-product/1/types/Article)
- [Byline](https://developer.nytimes.com/docs/articlesearch-product/1/types/Byline)
- [Headline](https://developer.nytimes.com/docs/articlesearch-product/1/types/Headline)
- [Keyword](https://developer.nytimes.com/docs/articlesearch-product/1/types/Keyword)
- [Multimedia](https://developer.nytimes.com/docs/articlesearch-product/1/types/Multimedia)
- [Person](https://developer.nytimes.com/docs/articlesearch-product/1/types/Person)

Let's try this again but for a specific time period. 

**For example, how would retrieve all the articles about the first two months of the George Floyd protests?**

We need to pass a dictionary to the `dates` argument which contains keys named "begin" and "end". Those two keys point to `datetime` objects that we'll use as time markers.

We're also going to use the `options` to filter and sort our results.


In [ ]:
#Set up start and end date objects
begin = datetime(2020, 5, 23)
end = datetime(2020, 7, 23)
#Create dictionary containing dates data
date_dict = {"begin":begin, "end":end}

#Create options dictionary
options_dict = {
#Sort from earliest to latest
        "sort": "oldest",
#Return only articles from New York Times, filters out other sources such as AP and Reuters
        "sources": [
            "New York Times"
        ],
#Return only straight-forward news in the form of articles
        "type_of_material": [
            "News Analysis", "News", "Article"
        ]
    }

articles = nyt.article_search(
    query = "George Floyd protest",
    results = 100,
    dates =date_dict,
    options = options_dict)

In [ ]:
#Grab first article and drop the multimedia key to reduce clutter
article = articles[0]
del article["multimedia"]

In [ ]:
#Check out results
article

#### Challenge 3: Article Searching

- Retrieve a set of articles for a query of your choice.
- Use a relevant time interval in constructing your `dates` dictionary
- Use "type_of_material" and "section_name" as keys for your `options` dictionary.
    - For "type_of_material" values refer to this [list](https://github.com/michadenheijer/pynytimes/blob/main/VALID_SEARCH_OPTIONS.md#type-of-material-values)
    - For "section_name" values refer to this [list](https://github.com/michadenheijer/pynytimes/blob/main/VALID_SEARCH_OPTIONS.md#section-name-values)

In [2]:
#solution

## 3. Data Analysis
*****

In this walk-through we will analyze articles and their data about the 2020 presidential election.

We are working with previously queried because it takes too much time actually to make the call. The code I used query and save the data can be found commented out below.

In [ ]:
# begin = datetime(2020, 9, 7)
# end = datetime(2020, 11, 7)
# date_dict = {"begin":begin, "end":end}

# options_dict = {
#         "sort": "oldest",
#         "sources": [
#             "New York Times",
#         ],
#         "type_of_material": [
#             "News Analysis", "News", "Article", "Editorial"
#         ]
#     }

# articles = nyt.article_search(
#     query = "presidential election",
#     results = 2500,
#     dates =date_dict,
#     options = options_dict)
# df = pd.json_normalize(articles)
# df.to_pickle("election2020_articles.pkl")

Load in the previously saved data

In [ ]:
df = pd.read_pickle("election2020_articles.pkl")
df.head()

In [ ]:
#Inspect metadata
df.info()

#### Sentiment Analysis

Let's track the sentiment of articles about the election over the two month time period. Sentiment analysis is a common task for using APIs but in order to save time from signing up for another API key we'll use the vadersentiment package instead.

From the [Vader Github Repo](https://github.com/cjhutto/vaderSentiment):

VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is *specifically attuned to sentiments expressed in social media*.

In [ ]:
#Install the vadersentiment library
!pip install vadersentiment

In [ ]:
#Import the Sentiment Analyzer object
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
#Initialize analyzer object
analyzer = SentimentIntensityAnalyzer()
#Calculate the polarity scores of the lead paragraph and save it in df
df["sentiment"] = df.lead_paragraph.apply(analyzer.polarity_scores)

In [ ]:
#Inspect the sentiment column
df.sentiment.head()

In [ ]:
#View single row
df.sentiment.iloc[0]

From Vader:

The ``compound`` score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate. 
 
  It is also useful for researchers who would like to set standardized thresholds for classifying sentences as either positive, neutral, or negative.  

Typical threshold values are:

 #. **positive sentiment**: ``compound`` score >=  0.05
 
 #. **neutral  sentiment**: (``compound`` score > -0.05) and (``compound`` score < 0.05)
 
 #. **negative sentiment**: ``compound`` score <= -0.05

In [ ]:
#Use only the compound score

In [ ]:
df["sentiment"] = df["sentiment"].apply(lambda x:x["compound"])

Let's get a sense of the distribution of scores by calculating some summary stats and visualizing the histogram.

In [ ]:
#Summary Stats
df.sentiment.describe()

In [ ]:
plt.style.use("ggplot")
df.sentiment.hist(bins = 20, figsize= (9, 7))
plt.xlabel("Sentiment Score")
plt.ylabel("Frequency");

#### Challenge 4. Answer questions about the data

- What are the 3 most positive and negative texts?
- Using the vader threshold for positive, neutral, and negative, how many article qualify for each of those labels?


In [3]:
#solution

**How did sentiment fluctuate over the course of the campaign?**

In [ ]:
#Create a panda series using the publication dates as the index and sentiment scores as the values
sentiment_ts = pd.Series(index= df.pub_date.tolist(), data = df.sentiment.tolist())

In [ ]:
#Resample aka groupby the data using d for daily and w for weekly
daily = sentiment_ts.resample("d").mean()
weekly = sentiment_ts.resample("w").mean()

In [ ]:
#Daily average sentiment of articles.
daily.plot(figsize = (11, 7))
plt.xlabel("Dates")
plt.ylabel("Sentiment Score");

In [ ]:
#Weekly average sentiment of articles.
weekly.plot(figsize = (11, 7))
plt.xlabel("Dates")
plt.ylabel("Sentiment Score");

### Bonus Section: Handling nested arrays of keywords


The Times has done us a favor in providing the named entities in the articles, thus relieveing us of having to do that task. However the data structure that it comes in can be tricky to handle. 

We've provided a short tutorial showing one way to cleanly extract keyword data.

In [ ]:
#Refer to a sample article's set of keywords
df.keywords.iloc[1]

We see a number of things here:
- Each article's keywords are laid out in a list of dictionaries.
- A dictionary tell us the name, type, ranking, and major of the keyword.
- The five types of keywords are: subject, persons, glocations, organizations, and creative_works
- The ordering of the list corresponds to the ranking.
- All articles do not all have the same number of rankings, ranges from 1 - 19.

We've created a function to extract keyword data based on the ranking. This function will be applied over the pandas series of keyword data.

In [ ]:

def rank_extractor(data, rank):
    #Empty list return non
    if data == []:
        return None
    #Iterate over the list of keywords until you reach the keyword corresponding with the ranking.
    for i in data:
        if i["rank"] == rank:
    #return just keyword and its type in a dictionary form
            return {"name":i["name"], "value":i["value"]}

In [ ]:
#Extract the first, second, and third keywords
rank1 = df.keywords.apply(lambda x: rank_extractor(x,1))
rank2 = df.keywords.apply(lambda x: rank_extractor(x,2))
rank3 = df.keywords.apply(lambda x: rank_extractor(x,3))

In [ ]:
#View results
rank1.head()

Convert the series containing dictionaries to a dataframe

In [ ]:
rank1 = rank1.apply(pd.Series)
rank2 = rank2.apply(pd.Series)
rank3 = rank3.apply(pd.Series)
rank1.head()

Voila, nice clean format. Now can we conduct some light analysis.

In [ ]:
#Most frequent type of keyword in ranking #1
rank1.name.value_counts()


In [ ]:
#Most common glocation in rank1
rank1.query("name == 'glocations'").value.value_counts().head()